# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

Date      id Warehouse   Shipments  \
0  Dec 14 2022  3:06PM  253134        10  SSR-000269   
1  Dec 14 2022  2:59PM  253133        10  CTF0011543   
2  Dec 14 2022  2:49PM  253130        10   MSP216761   
3  Dec 14 2022  2:49PM  253130        10   MSP216762   
4  Dec 14 2022  2:49PM  253130        10   MSP216763   
5  Dec 14 2022  2:49PM  253130        10   MSP216764   
6  Dec 14 2022  2:49PM  253130        10   MSP216765   
7  Dec 14 2022  2:49PM  253130        10   MSP216766   
8  Dec 14 2022  2:49PM  253130        10   MSP216767   
9  Dec 14 2022  2:49PM  253130        10   MSP216768   

                   Customer ShipmentStatus  
0  Senseonics, Incorporated       Released  
1            Citieffe, Inc.       Released  
2          Methapharm, Inc.       Released  
3          Methapharm, Inc.       Released  
4          Methapharm, Inc.       Released  
5          Methapharm, Inc.       Released  
6          Methapharm, Inc.       Released  
7          Methapharm, Inc.       Released  
8          Methapharm, Inc.       Released  
9          Methapharm, Inc.       Released

In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

id ShipmentStatus  Shipments
41  253130       Released         23
42  253131       Released          1
43  253132       Released          1
44  253133       Released          1
45  253134       Released          1

In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus  Completed  Executing  Released
id                                            
253130                NaN        NaN      23.0
253131                NaN        NaN       1.0
253132                NaN        NaN       1.0
253133                NaN        NaN       1.0
253134                NaN        NaN       1.0

In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus  Completed  Executing  Released
id                                            
253051                5.0        2.0       8.0
253053               14.0        1.0      10.0
253060               16.0        9.0       7.0
253062                9.0        1.0       2.0
253064                0.0        0.0      23.0

In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus  Completed  Executing  Released
id                                            
253051                  5          2         8
253053                 14          1        10
253060                 16          9         7
253062                  9          1         2
253064                  0          0        23

In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus      id  Completed  Executing  Released
0               253051          5          2         8
1               253053         14          1        10
2               253060         16          9         7
3               253062          9          1         2
4               253064          0          0        23

In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus      id Completed Executing Released
0               253051         5         2        8
1               253053        14         1       10
2               253060        16         9        7
3               253062         9         1        2
4               253064                           23

In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

Date      id Warehouse                        Customer
0   Dec 14 2022  3:06PM  253134        10        Senseonics, Incorporated
1   Dec 14 2022  2:59PM  253133        10                  Citieffe, Inc.
2   Dec 14 2022  2:49PM  253130        10                Methapharm, Inc.
25  Dec 14 2022  2:48PM  253132        21               NBTY Global, Inc.
26  Dec 14 2022  2:48PM  253131        21               NBTY Global, Inc.
27  Dec 14 2022  2:43PM  253129        10        Senseonics, Incorporated
28  Dec 14 2022  2:38PM  253128        10    Snap Medical Industries, LLC
30  Dec 14 2022  2:34PM  253127        19             AdvaGen Pharma, Ltd
46  Dec 14 2022  2:28PM  253126        19         GUSA Granules USA, Inc.
47  Dec 14 2022  2:24PM  253125        16  Sartorius Bioprocess Solutions

In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

Date      id Warehouse                        Customer  \
0  Dec 14 2022  3:06PM  253134        10        Senseonics, Incorporated   
1  Dec 14 2022  2:59PM  253133        10                  Citieffe, Inc.   
2  Dec 14 2022  2:49PM  253130        10                Methapharm, Inc.   
3  Dec 14 2022  2:48PM  253132        21               NBTY Global, Inc.   
4  Dec 14 2022  2:48PM  253131        21               NBTY Global, Inc.   
5  Dec 14 2022  2:43PM  253129        10        Senseonics, Incorporated   
6  Dec 14 2022  2:38PM  253128        10    Snap Medical Industries, LLC   
7  Dec 14 2022  2:34PM  253127        19             AdvaGen Pharma, Ltd   
8  Dec 14 2022  2:28PM  253126        19         GUSA Granules USA, Inc.   
9  Dec 14 2022  2:24PM  253125        16  Sartorius Bioprocess Solutions   

  Completed Executing Released  
0                            1  
1                            1  
2                           23  
3                            1  
4                            1  
5                            1  
6                            2  
7                           16  
8                            1  
9                            1

In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

Date      id Warehouse                        Customer  \
0  Dec 14 2022  3:06PM  253134        10        Senseonics, Incorporated   
1  Dec 14 2022  2:59PM  253133        10                  Citieffe, Inc.   
2  Dec 14 2022  2:49PM  253130        10                Methapharm, Inc.   
3  Dec 14 2022  2:48PM  253132        21               NBTY Global, Inc.   
4  Dec 14 2022  2:48PM  253131        21               NBTY Global, Inc.   
5  Dec 14 2022  2:43PM  253129        10        Senseonics, Incorporated   
6  Dec 14 2022  2:38PM  253128        10    Snap Medical Industries, LLC   
7  Dec 14 2022  2:34PM  253127        19             AdvaGen Pharma, Ltd   
8  Dec 14 2022  2:28PM  253126        19         GUSA Granules USA, Inc.   
9  Dec 14 2022  2:24PM  253125        16  Sartorius Bioprocess Solutions   

  Released Executing Completed  
0        1                      
1        1                      
2       23                      
3        1                      
4        1                      
5        1                      
6        2                      
7       16                      
8        1                      
9        1

In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

Date      id Warehouse                  Customer Released  \
0  Dec 14 2022  3:06PM  253134        10  Senseonics, Incorporated        1   
1  Dec 14 2022  2:59PM  253133        10            Citieffe, Inc.        1   
2  Dec 14 2022  2:49PM  253130        10          Methapharm, Inc.       23   
3  Dec 14 2022  2:48PM  253132        21         NBTY Global, Inc.        1   
4  Dec 14 2022  2:48PM  253131        21         NBTY Global, Inc.        1   

  Executing Completed  
0                      
1                      
2                      
3                      
4

In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

Date      id Warehouse                  Customer  Released  \
0  Dec 14 2022  3:06PM  253134        10  Senseonics, Incorporated       1.0   
1  Dec 14 2022  2:59PM  253133        10            Citieffe, Inc.       1.0   
2  Dec 14 2022  2:49PM  253130        10          Methapharm, Inc.      23.0   
3  Dec 14 2022  2:48PM  253132        21         NBTY Global, Inc.       1.0   
4  Dec 14 2022  2:48PM  253131        21         NBTY Global, Inc.       1.0   

   Executing  Completed  
0        NaN        NaN  
1        NaN        NaN  
2        NaN        NaN  
3        NaN        NaN  
4        NaN        NaN

In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

Date      id Warehouse                  Customer  Released  \
0  Dec 14 2022  3:06PM  253134        10  Senseonics, Incorporated       1.0   
1  Dec 14 2022  2:59PM  253133        10            Citieffe, Inc.       1.0   
2  Dec 14 2022  2:49PM  253130        10          Methapharm, Inc.      23.0   
3  Dec 14 2022  2:48PM  253132        21         NBTY Global, Inc.       1.0   
4  Dec 14 2022  2:48PM  253131        21         NBTY Global, Inc.       1.0   

   Executing  Completed  
0        0.0        0.0  
1        0.0        0.0  
2        0.0        0.0  
3        0.0        0.0  
4        0.0        0.0

In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

id  Released  Executing  Completed
Warehouse                                         
10         4302792     118.0        8.0       13.0
15         1518509      65.0        3.0       19.0
16          253125       1.0        0.0        0.0
19         2024803      32.0        9.0       16.0
20          506205       2.0        0.0        0.0
21          506263       2.0        0.0        0.0

In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

Warehouse       id  Released  Executing  Completed
0        10  4302792     118.0        8.0       13.0
1        15  1518509      65.0        3.0       19.0
2        16   253125       1.0        0.0        0.0
3        19  2024803      32.0        9.0       16.0
4        20   506205       2.0        0.0        0.0
5        21   506263       2.0        0.0        0.0

In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

Warehouse  Released  Executing  Completed
0        10     118.0        8.0       13.0
1        15      65.0        3.0       19.0
2        16       1.0        0.0        0.0
3        19      32.0        9.0       16.0
4        20       2.0        0.0        0.0
5        21       2.0        0.0        0.0

In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

Warehouse    Status  Value
0        10  Released  118.0
1        15  Released   65.0
2        16  Released    1.0
3        19  Released   32.0
4        20  Released    2.0

In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse     10    15   16    19   20   21
Status                                     
Completed   13.0  19.0  0.0  16.0  0.0  0.0
Executing    8.0   3.0  0.0   9.0  0.0  0.0
Released   118.0  65.0  1.0  32.0  2.0  2.0

In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse     Status     10    15   16    19   20   21
0          Completed   13.0  19.0  0.0  16.0  0.0  0.0
1          Executing    8.0   3.0  0.0   9.0  0.0  0.0
2           Released  118.0  65.0  1.0  32.0  2.0  2.0

In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

Status     10    15   16    19   20   21
0  Completed   13.0  19.0  0.0  16.0  0.0  0.0
1  Executing    8.0   3.0  0.0   9.0  0.0  0.0
2   Released  118.0  65.0  1.0  32.0  2.0  2.0

In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()